# Heuristic model


##### We have run this version on kaggle for convenient:
https://www.kaggle.com/code/kientranhust/heuristic-models-based-on-similarity-of-words

### Pretrained model: Word2Vec
https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300

In [45]:
import json
import os
import json
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk

## Run on kaglle
# nltk.download('wordnet', download_dir = "/kaggle/working/nltk_data")
# nltk.download('averaged_perceptron_tagger',download_dir = "/kaggle/working/nltk_data") 

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import wordnet
np.random.seed(2023)
import random
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
nltk.data.path.append('/kaggle/working/nltk_data')

# Now, you should be able to use WordNet without any issues
from nltk.corpus import wordnet
!mkdir /kaggle/working/nltk_data

mkdir: cannot create directory ‘/kaggle/working/nltk_data’: File exists


In [3]:
#First, you're going to need to import wordnet:
import nltk

nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

# Preprocessing data

In [5]:
def lemmatize_stemming(text):
    result = WordNetLemmatizer().lemmatize(text, pos='v')
    result = WordNetLemmatizer().lemmatize(result, pos='n')
    result = WordNetLemmatizer().lemmatize(result, pos='a')
    return result

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return " ".join(result)

In [6]:
def count_word_appearances(sentence, wordDict):
    # Split the sentence into words
    words = sentence.split()


    # Iterate through each word in the list
    for word in words:
        # Remove punctuation and convert to lowercase for case-insensitive counting
        word = word.strip('.,!?"\'').lower()

        wordDict[word] = wordDict.get(word, 0) + 1

    return wordDict



In [7]:
## Wrapping into function
def mostFrequentWord(content = None, path = None):
    myDict = {}
    if path:
        with open(path, 'r') as file:
            data = json.load(file)
        totalDoc = ""
        countDrop = 0
        for key in data.keys():
            if data[key]['Content'] != "None":
                totalDoc += " " + preprocess(data[key]['Content'])
            else:
                countDrop += 1
        print(f"Number of articles with no content is {countDrop}")
    else: 
        totalDoc = preprocess(content)
    myDict = count_word_appearances(totalDoc, myDict)
    sortedDict = sorted(myDict.items(), key = lambda x:x[1], reverse = True)
    return sortedDict

In [8]:
#Check whether a word is a noun or not
def isNoun(word):
    words = [word]
    ans = nltk.pos_tag(words) 
    val = ans[0][1]
    if(val == 'NN' or val == 'NNS' or val == 'NNPS' or val == 'NNP'): 
        return True 
    return False

In [9]:
## Function to get top 100
def getTopWord(customDict, top_n = 100):
    topNoun = []
    countt = 0
    for word in customDict:
        if (isNoun(word[0])):
            topNoun.append(word[0])
            countt += 1
        if countt == top_n:
            break
    return topNoun

In [10]:
top_n = 100

In [11]:
# with open("/kaggle/input/the-telegraph/Business.json", 'r') as file:
#     businessData = json.load(file)
# with open('/kaggle/input/the-telegraph/Culture.json', 'r') as file:
#     cultureData = json.load(file)
# with open('/kaggle/input/the-telegraph/Education.json', 'r') as file:
#     educationData = json.load(file)
# with open('/kaggle/input/the-telegraph/Environment.json', 'r') as file:
#     environmentData = json.load(file)
# with open('/kaggle/input/the-telegraph/Football.json', 'r') as file:
#     footballData = json.load(file)
# with open('/kaggle/input/the-telegraph/Health.json', 'r') as file:
#     healthData = json.load(file)
# with open('/kaggle/input/the-telegraph/Politics.json', 'r') as file:
#     politicsData = json.load(file)
# with open('/kaggle/input/the-telegraph/Science.json', 'r') as file:
#     scienceData = json.load(file)
# with open('/kaggle/input/the-telegraph/Style.json', 'r') as file:
#     styleData = json.load(file)
# with open('/kaggle/input/the-telegraph/Technology.json', 'r') as file:
#     technologyData = json.load(file)
# with open('/kaggle/input/the-telegraph/Travel.json', 'r') as file:
#     travelData = json.load(file)
with open("data/Business.json", 'r') as file:
    businessData = json.load(file)
with open('data/Culture.json', 'r') as file:
    cultureData = json.load(file)
with open('data/Education.json', 'r') as file:
    educationData = json.load(file)
with open('data/Environment.json', 'r') as file:
    environmentData = json.load(file)
with open('data/Football.json', 'r') as file:
    footballData = json.load(file)
with open('data/Health.json', 'r') as file:
    healthData = json.load(file)
with open('data/Politics.json', 'r') as file:
    politicsData = json.load(file)
with open('data/Science.json', 'r') as file:
    scienceData = json.load(file)
with open('data/Style.json', 'r') as file:
    styleData = json.load(file)
with open('data/Technology.json', 'r') as file:
    technologyData = json.load(file)
with open('data/Travel.json', 'r') as file:
    travelData = json.load(file)

## Business

In [12]:
## Read data
with open("data/Business.json", 'r') as file:
    businessData = json.load(file)
with open("data/Company.json", "r") as file:
    companyData = json.load(file)
with open("data/Market.json", "r") as file:
    marketData = json.load(file)
    

In [13]:
totalDoc = ""
countDrop = 0

for key in businessData.keys():
    if businessData[key]['Content'] != "None":
        totalDoc += " " + preprocess(businessData[key]['Content'])
    else:
        countDrop += 1
for key in companyData.keys():
    if companyData[key]['Content'] != "None":
        totalDoc += " " + preprocess(companyData[key]['Content'])
    else:
        countDrop += 1
for key in marketData.keys():
    if marketData[key]['Content'] != "None":
        totalDoc += " " + preprocess(marketData[key]['Content'])
    else:
        countDrop += 1
print("Number of articles with None content is: " + str(countDrop))
print("Done!")

Number of articles with None content is: 12
Done!


In [14]:
businessDict = mostFrequentWord(content = totalDoc)
topBusiness = getTopWord(businessDict, top_n)
len(topBusiness)

100

# Health
We merge category health and global security health since they are much similar

In [15]:
with open("data/Health.json", 'r') as file:
    healthData = json.load(file)
    
with open("data/GlobalHealthSecurity.json") as file:
    healthSecurityData = json.load(file)

In [16]:
healthDoc = ""
countDrop = 0

In [17]:
for key in healthData.keys():
    if healthData[key]['Content'] != "None":
        healthDoc += " " + preprocess(healthData[key]['Content'])
    else:
        countDrop += 1
for key in healthSecurityData.keys():
    if healthSecurityData[key]['Content'] != "None":
        healthDoc += " " + preprocess(healthSecurityData[key]['Content'])
    else:
        countDrop += 1
print("Done")
print(f"Number of articles with None content is: {countDrop}")
print(f"Number of characters is {len(healthDoc)}")

Done
Number of articles with None content is: 124
Number of characters is 56519694


In [18]:
healthDict = mostFrequentWord(content = healthDoc)
topHealth = getTopWord(healthDict, top_n)
print(len(topHealth))

100


# Football

In [19]:
footballDict = mostFrequentWord(path = "data/Football.json")

Number of articles with no content is 21


In [20]:
topFootball = getTopWord(footballDict, top_n)

In [21]:
len(topFootball)

100

# Polictics

In [22]:
politicsDict = mostFrequentWord(path = "data/Politics.json")

Number of articles with no content is 39


In [23]:
topPolitics = getTopWord(politicsDict, top_n)

In [24]:
topPolitics

['minister',
 'government',
 'johnson',
 'people',
 'party',
 'year',
 'prime',
 'tory',
 'labour',
 'tell',
 'secretary',
 'time',
 'sunak',
 'boris',
 'work',
 'think',
 'plan',
 'need',
 'country',
 'week',
 'truss',
 'leader',
 'cent',
 'support',
 'want',
 'public',
 'election',
 'home',
 'right',
 'vote',
 'change',
 'include',
 'rule',
 'office',
 'face',
 'rishi',
 'ask',
 'street',
 'report',
 'number',
 'telegraph',
 'month',
 'claim',
 'policy',
 'cost',
 'covid',
 'credit',
 'cabinet',
 'warn',
 'call',
 'keir',
 'britain',
 'house',
 'issue',
 'member',
 'ukraine',
 'police',
 'today',
 'help',
 'look',
 'leadership',
 'rise',
 'decision',
 'thing',
 'mean',
 'energy',
 'health',
 'place',
 'business',
 'chancellor',
 'increase',
 'deal',
 'question',
 'meet',
 'force',
 'case',
 'service',
 'committee',
 'source',
 'lord',
 'state',
 'ireland',
 'official',
 'spend',
 'continue',
 'point',
 'brexit',
 'speak',
 'hold',
 'world',
 'crisis',
 'campaign',
 'write',
 'concern

# Science

In [25]:
scienceDict = mostFrequentWord(path = "data/Science.json")

Number of articles with no content is 46


In [26]:
topScience = getTopWord(scienceDict, top_n)

# Education

In [27]:
educationDict = mostFrequentWord(path = "data/Education.json")

Number of articles with no content is 1


In [28]:
topEducation = getTopWord(educationDict, top_n)

# Environment

In [29]:
environmentDict = mostFrequentWord(path = "data/Environment.json")

Number of articles with no content is 10


In [30]:
topEnvironment = getTopWord(environmentDict, top_n)

# Technology

In [31]:
technologyDict = mostFrequentWord(path = "data/Technology.json")

Number of articles with no content is 78


In [32]:
topTechnology = getTopWord(technologyDict, top_n)

# Culture

In [33]:
cultureDict = mostFrequentWord(path = "data/Culture.json")

Number of articles with no content is 5961


In [34]:
topCulture = getTopWord(cultureDict, top_n)

In [35]:
topCulture

['year',
 'time',
 'film',
 'credit',
 'work',
 'people',
 'think',
 'play',
 'write',
 'life',
 'tell',
 'thing',
 'world',
 'want',
 'look',
 'music',
 'book',
 'love',
 'star',
 'feel',
 'story',
 'woman',
 'series',
 'song',
 'week',
 'london',
 'turn',
 'child',
 'right',
 'need',
 'start',
 'include',
 'call',
 'record',
 'point',
 'family',
 'home',
 'mean',
 'artist',
 'character',
 'actor',
 'read',
 'show',
 'place',
 'house',
 'album',
 'theatre',
 'audience',
 'watch',
 'dance',
 'change',
 'talk',
 'night',
 'band',
 'friend',
 'begin',
 'john',
 'release',
 'director',
 'bring',
 'sound',
 'moment',
 'month',
 'role',
 'line',
 'face',
 'scene',
 'help',
 'hear',
 'drama',
 'idea',
 'death',
 'try',
 'kind',
 'stage',
 'novel',
 'father',
 'word',
 'hand',
 'head',
 'ask',
 'rock',
 'fact',
 'school',
 'history',
 'experience',
 'speak',
 'career',
 'cast',
 'day',
 'wasn',
 'performance',
 'writer',
 'mother',
 'production',
 'public',
 'break',
 'picture',
 'course',
 '

# Travel

In [36]:
travelDict = mostFrequentWord(path = "data/Travel.json")

Number of articles with no content is 38


In [37]:
topTravel = getTopWord(travelDict, top_n)

In [38]:
topTravel

['hotel',
 'travel',
 'credit',
 'year',
 'holiday',
 'book',
 'country',
 'time',
 'include',
 'night',
 'test',
 'rat',
 'expert',
 'people',
 'room',
 'city',
 'island',
 'place',
 'offer',
 'beach',
 'flight',
 'covid',
 'restaurant',
 'read',
 'world',
 'stay',
 'home',
 'week',
 'provide',
 'visit',
 'family',
 'look',
 'check',
 'list',
 'resort',
 'need',
 'telegraph',
 'house',
 'summer',
 'price',
 'cruise',
 'getty',
 'government',
 'view',
 'review',
 'quarantine',
 'trip',
 'day',
 'hour',
 'park',
 'town',
 'work',
 'area',
 'walk',
 'case',
 'return',
 'month',
 'build',
 'plan',
 'destination',
 'start',
 'traveller',
 'south',
 'tour',
 'availability',
 'london',
 'airport',
 'restriction',
 'tourist',
 'passenger',
 'water',
 'head',
 'cost',
 'pool',
 'want',
 'guide',
 'experience',
 'century',
 'lockdown',
 'think',
 'feel',
 'village',
 'cent',
 'mean',
 'thing',
 'visitor',
 'rule',
 'number',
 'tell',
 'star',
 'child',
 'mountain',
 'change',
 'mile',
 'service

# Style

In [39]:
styleDict = mostFrequentWord(path = "data/Style.json")

Number of articles with no content is 17


In [40]:
topStyle = getTopWord(styleDict, top_n)

In [41]:
topStyle

['look',
 'wear',
 'dress',
 'year',
 'credit',
 'fashion',
 'time',
 'work',
 'style',
 'brand',
 'want',
 'colour',
 'think',
 'feel',
 'woman',
 'design',
 'hair',
 'home',
 'house',
 'need',
 'people',
 'love',
 'thing',
 'getty',
 'clothe',
 'shirt',
 'piece',
 'london',
 'suit',
 'collection',
 'designer',
 'skin',
 'include',
 'post',
 'world',
 'right',
 'week',
 'room',
 'create',
 'start',
 'instagram',
 'life',
 'pair',
 'beauty',
 'coat',
 'wardrobe',
 'image',
 'trouser',
 'family',
 'jacket',
 'mean',
 'summer',
 'face',
 'place',
 'help',
 'shop',
 'queen',
 'month',
 'light',
 'hand',
 'tell',
 'royal',
 'princess',
 'outfit',
 'sell',
 'offer',
 'wed',
 'product',
 'change',
 'luxury',
 'model',
 'share',
 'street',
 'jean',
 'skirt',
 'telegraph',
 'print',
 'gold',
 'point',
 'view',
 'watch',
 'boot',
 'shoe',
 'line',
 'price',
 'body',
 'friend',
 'turn',
 'space',
 'favourite',
 'duchess',
 'leather',
 'star',
 'day',
 'spend',
 'diamond',
 'size',
 'season',
 'b

# Model

In [42]:
# Load Word2Vec model
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)

In [43]:
#Create wordDict to store K characteristics words of each category
wordDict = []
wordDict.append(topBusiness)
wordDict.append(topCulture)
wordDict.append(topEducation)
wordDict.append(topEnvironment)
wordDict.append(topFootball)
wordDict.append(topHealth)
wordDict.append(topPolitics)
wordDict.append(topScience)
wordDict.append(topStyle)
wordDict.append(topTechnology)
wordDict.append(topTravel)


In [44]:
#Function to get category of an article after we extract its frequent words
def getCategory(key):
    scoreLst = []
    for wordLst in wordDict:
        countt = 0
        total = 0
        for word in key:
            for word2 in wordLst:
                try:
                    total += model.similarity(word[0], word2)
                    countt += 1
                except: 
                    continue

        scoreLst.append(total/countt)
    argmax = scoreLst.index(max(scoreLst))
    return argmax, scoreLst

# Test

In [48]:
# Funtion to test
def test(test_size = 50):
    labels = []
    predictions = []
    for data in dataList:
        testSize = list(range(1, len(data.keys())))
        random_elements = random.sample(testSize, test_size)       
        countt = 0
        noneContent = 0
        for i in random_elements:
            docs = data[f"{i}"]['Content']
            if docs == "None":
                noneContent += 1
                continue
            else:
                docWord = mostFrequentWord(content = preprocess(docs))
                key = docWord[:50]
                ans = getCategory(key)
                if ans[0] == dataList.index(data):
                    countt += 1
            labels.append(dataList.index(data))
            predictions.append(ans[0])
    return labels, predictions

In [46]:
dataList = [businessData, cultureData, educationData, environmentData, footballData, healthData, politicsData, scienceData, styleData, technologyData, travelData]
name = ['Business', 'Culture', 'Education', 'Environment', 'Football', 'Health', 'Politics', 'Science', 'Style', 'Technology', 'Travel']


In [51]:
label, prediction = test(10)

In [52]:
print(classification_report(prediction, label, target_names = name))

              precision    recall  f1-score   support

    Business       0.89      0.73      0.80        11
     Culture       1.00      0.50      0.67        14
   Education       1.00      0.91      0.95        11
 Environment       0.70      0.64      0.67        11
    Football       0.90      0.82      0.86        11
      Health       0.20      0.40      0.27         5
    Politics       0.60      0.46      0.52        13
     Science       0.50      0.62      0.56         8
       Style       0.90      0.75      0.82        12
  Technology       0.30      0.75      0.43         4
      Travel       0.50      0.83      0.62         6

    accuracy                           0.67       106
   macro avg       0.68      0.67      0.65       106
weighted avg       0.76      0.67      0.69       106



# Inference

In [53]:
def infer(docs):
    key = mostFrequentWord(content = preprocess(docs))
    ind, score = getCategory(key)
    print(f"This article is in {cateName[ind]} category")
    return ind, score

In [54]:
cateName = ['Business', 'Culture', 'Education', 'Environment', 'Football', 'Health', 'Politics', 'Science', 'Style', 'Technology', 'Travel']


In [55]:
### Enter content of an article

docs = """In the old days, there were the FAANGs, the five big US tech stocks that dominated the investment landscape – Facebook (now Meta), Amazon, Apple, Netflix and Google (now Alphabet). That picture is now out of date. Say hello instead to what is variously called the Super Seven or the Magnificent Seven – four of the above (the dropout being Netflix) plus Microsoft, Tesla and the chip-maker Nvidia. This group’s domination is the stock market story of 2023.

The chart below is “one for the ages”, says Duncan Lamont, the head of strategic research at the fund manager Schroders. It shows how, even if you invest via one of the broadest and most widely used “global” stock market indices, you will end up with a portfolio that is very American and very skewed towards US tech.

The index is the MSCI All Country World Index (ACWI), which covers approximately 85% of “the global investable equity opportunity”, as the compilers put it, by measuring almost 3,000 large and mid-sized companies in 23 developed markets and 24 emerging ones. The bigger a company becomes in value, the greater its weighting in the index.The numbers have become so astonishing, in part, because of what is shown in the second chart. Up to last week, the group of seven has risen in value by 74% in 2023. The rest of the world’s equities, within the same ACWI index, have managed 12%. If your portfolio did not include the Magnificent Seven in 2023, it was hard to keep up.

Is this degree of concentration healthy? It’s certainly unprecedented. Thanks to the whoosh from the seven during 2023, US stocks now account for 63% of the supposedly global ACWI. Even in the go-go days of the Japanese economic miracle, the country accounted for only 44% of the same index. “The US has far exceeded the level of concentration of Japan in the 1980s, which everyone thought was extreme at the time,” says Lamont.Yet it would be hard to argue that the rise of the seven has been fuelled by the type of wild speculation that created the turn-of-the-century dotcom bubble. The 240% rise in Nvidia’s stock price this year may or may not be overdone, but it’s undeniable that the company’s order book for computer chips is booming as the artificial intelligence (AI) revolution arrives.

It would also be wrong to think of the seven as entirely alike. All have leading positions in growing markets and Amazon, Google and Microsoft have big cloud services divisions. But Amazon’s retail division has little in common with Google’s search business and Microsoft’s core software business is different again. All may benefit from AI, which helps to explain the stock market’s renewed love affair with technology in 2023 after a heavy “down” year in 2022, but the degrees will differ. Tesla remains, primarily, a maker of electric vehicles."""

In [56]:
infer(docs)

This article is in Business category


(0,
 [0.09341262750253389,
  0.08102950812606095,
  0.0786857995416624,
  0.0863375881468453,
  0.07636080529179111,
  0.07953556176170777,
  0.08052936722600523,
  0.08117637550019245,
  0.081362155994441,
  0.09101626295824473,
  0.07722635968033263])